In [1]:
# share to here:
# nlf-automate-rides-service-acc@nlf-automate-rides.iam.gserviceaccount.com
link_to_sheet = "https://docs.google.com/spreadsheets/d/1HoSoVgSTPdy2VV7zRvZMhtQX77LgRZ13DXvOQSluSNQ/edit?gid=1076432338#gid=1076432338"
sheet_name = "Form Responses RESET"

In [2]:
# The name of the csv file that you want to read. It should be in the same file level of this code
FORM_NAME = "form.csv"

'''
Column Form Titles - make sure these match the questions on the form.
'''
NAME_COLUMN = "Name"
PICKUP_COLUMN = "Where would you like to be picked up?"
SERVICE_TYPE_COLUMN = "Which service are you attending?"
AFTER_SERVICE_PLANS_COLUMN = "Preferred after church plans?"
IS_DRIVER_COLUMN = "Are you a driver?"
OC_ADDRESS = "Off Campus address" #This is either "Additional Info" or "Off Campus Address" depending on the form question

'''
For the "Where would you like to be pickedup question", the choices for the question should be in here.
There should be a constant and an entry in the dictionary below (should be in both).
'''
NORTH_STOP_NAME = "North (Brown, Duncan, Jones, Martel, McMurtry)"
SOUTH_STOP_NAME = "South (Baker, Hanszen, Lovett, Sid Richardson, Wiess, Will Rice)"
LIFETOWER_STOP_NAME = "Life tower"

location_to_address = {
    NORTH_STOP_NAME: "1601 Rice Boulevard, Houston, TX 77005",
    SOUTH_STOP_NAME: "6320 Main St, Houston, TX 77005",
    LIFETOWER_STOP_NAME: "6919 Main St, Houston, TX 77030",
}

BACK_HOME_PLAN = "Back home 💙"
LUNCH_PLAN = "Lunch 💛"

In [3]:
# Default number of passengers the driver's car can hold.
PASSENGER_LIMIT = 4

'''
This is where you specify the maximum amount of seats a driver will have for a week instead of the default 4.
'''
AMOUNT_SEATS_CHANGE = {
    # "Matthew Ahn": 2,
}

'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders_to = {
    # "Jonathan Mak": {"shayla Nguyen", "Pedro Flores-Teran"},
    # "AZ Ellis": {"Pedro Flores-Teran"}
}

'''
Assign riders who should ride together
'''
rider_groups_to = [
    # {"Khang Le", "seojin Kwon"}
]


'''Back from church'''

'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders_back = {
    # "Jonathan Mak": {"Grace Kwon", "Seojin Kwon"},
    # "AZ Ellis": {"Pedro Flores-Teran"}
}

'''
# Assign riders who should ride together
'''
rider_groups_back = [
    # {"Khang Le", "Aaron duong"}
]

In [4]:
# Change the hex colors for the Excel here. Make sure it's 8 characters 
# (Add F's in the front if the hex is elss than 8 charactesr)
location_colors = {
    "North": "FFd9ead3",     # light green 3
    "South": "FF93CCEA",     # Light Cornflower Blue 3 
    "Off": "FFFFFFED",       # light yellow
    "Life": "fff4cccc",      # green
    BACK_HOME_PLAN: "FFD9D2E9",      # light purple
    "RJM": "FFEAD1DC",       # light pink
    LUNCH_PLAN: "FFFCE5CD",     # light orange
    "Flexible 💚": "FFCFE2F3",        # light blue
    "Refreshments": "FFB6D7A8",
    "NLK 🧡": "FFFFCCA4",
}

------------------------------------

**THE BOTTOM IS CODE THAT SHOULD NOT BE TOUCHED. The parameters above are stuff you can stuff without breaking.**

------------------------------------

In [5]:
import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import random
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
from datetime import datetime
import time
from zoneinfo import ZoneInfo
import glob
import copy

import folium
from folium import PolyLine
from folium.plugins import PolyLineTextPath
from collections import defaultdict
import matplotlib.colors as mcolors
import math

import gspread
from oauth2client.service_account import ServiceAccountCredentials

from dotenv import load_dotenv
import os

import matplotlib.pyplot as plt
from collections import defaultdict

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
start_time = time.time()

# Load environment variables from .env file
load_dotenv(override=True)

# Access the key
JSON_KEY_PATH = os.getenv("JSON_KEY_PATH")

# Define the scope and authenticate
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(JSON_KEY_PATH, scope)
client = gspread.authorize(creds)

# Open the sheet by URL or key
sheet = client.open_by_url(link_to_sheet)
worksheet = sheet.worksheet(sheet_name)
# print(worksheet)
values = worksheet.get_all_values()  # Raw values (no header check)
df = pd.DataFrame(values[1:], columns=values[0])  # Manually assign first row as headers

# print(len(df))
# print(df.tail())

In [ ]:
# df = pd.read_csv(FORM_NAME)

In [ ]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans, address):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans
        self.long_lat_pair = ()
        self.address = address

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

class Rider:
    def __init__(self, name, pickup_location, service_type, plans, address):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans
        self.long_lat_pair = ()
        self.address = address


    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [ ]:
coords_to_dist = dict()
drivers = set()
riders = set()

def dist(coord1, coord2):
    if (coord1, coord2) in coords_to_dist or (coord2, coord1) in coords_to_dist:
        return coords_to_dist[(coord1, coord2)]
    
    miles_distance = geodesic(coord1, coord2).miles

    coords_to_dist[(coord1, coord2)] = miles_distance
    coords_to_dist[(coord2, coord1)] = miles_distance

    return miles_distance

def route_cost(start, waypoints, destination):
    total = 0
    curr = start
    for wp in waypoints:
        total += dist(curr, wp.long_lat_pair)
        # print("what is wp", wp.name, total, curr)
        curr = wp.long_lat_pair
    total += dist(curr, destination)
    return total

In [ ]:
def assign_whitelisted_groups(drivers, riders, driver_required_riders, rider_groups):
    assignments = defaultdict(list)
    unassigned_riders = set(riders)

    # Normalize names to lowercase for case-insensitive matching
    name_to_driver = {d.name.lower(): d for d in drivers}
    name_to_rider = {r.name.lower(): r for r in riders}
    remaining_drivers = set(drivers)

    # Normalize rider_groups
    normalized_rider_groups = [[name.lower() for name in group] for group in rider_groups]

    # Normalize driver_required_riders
    normalized_driver_required = {
        driver_name.lower(): [rider_name.lower() for rider_name in rider_list]
        for driver_name, rider_list in driver_required_riders.items()
    }

    # Step 1: Assign rider-to-rider whitelist groups
    for group in normalized_rider_groups:
        group_objs = [name_to_rider[name] for name in group if name in name_to_rider]

        if len(group_objs) < 2:
            continue  # Discard group if fewer than 2 valid riders

        if not all(r in unassigned_riders for r in group_objs):
            continue  # Skip if someone already assigned

        for driver in list(remaining_drivers):
            if driver.amount_seats >= len(group_objs):
                assignments[driver].extend(group_objs)
                # Remove original rider objects from unassigned_riders
                for r in group_objs:
                    unassigned_riders.discard(r)
                driver.amount_seats -= len(group_objs)
                if driver.amount_seats == 0:
                    remaining_drivers.discard(driver)
                break

    # Step 2: Assign driver → rider whitelist
    for driver_name, required_names in normalized_driver_required.items():
        driver = name_to_driver.get(driver_name)
        if not driver:
            continue

        group_objs = [name_to_rider[name] for name in required_names if name in name_to_rider]
        print([x.name for x in group_objs])

        if not group_objs:
            continue  # Skip if no valid riders

        if not all(r in unassigned_riders for r in group_objs):
            continue  # Skip if someone already assigned

        if driver.amount_seats >= len(group_objs):
            assignments[driver].extend(group_objs)
            for r in group_objs:
                unassigned_riders.discard(r)  # Correctly removing the full Rider object
            driver.amount_seats -= len(group_objs)
            if driver.amount_seats == 0:
                remaining_drivers.discard(driver)

    return assignments, list(remaining_drivers), unassigned_riders

In [ ]:
# Initialize the geocoder
# geolocator = Nominatim(user_agent="ride_assignment_NLF")

from geopy.geocoders import GoogleV3

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
geolocator = GoogleV3(api_key=GOOGLE_API_KEY)

# Cache for distance: address → (lat, long)
address_coords = {}
oc_people_w_invalid_address = set()

def geocode_address(address):
    print("GEOCODE_ADDRESS", "address:", address)
    if address in address_coords:
        return address_coords[address]
    try:
        # Avoid appending "Houston" if it's already in the address
        if "houston" not in address.lower():
            full_address = f"{address}, houston, USA"
        else:
            full_address = f"{address}, USA"

        # for google
        location = geolocator.geocode(full_address, region="us", timeout=5)

        # for nomatin
        # location = geolocator.geocode(full_address, country_codes="us", timeout=5)
        print("Location returned by API for", full_address, "is", location, "with coord", (location.latitude, location.longitude))

        if location:
            coord = (location.latitude, location.longitude)
            address_coords[address] = coord
            return coord
            
    except Exception as e:
        print(f"Geocoding failed for '{address}': {e}")
        return e

    return True


In [ ]:
# Local optimization: Optimize locally for a given 'detour' heuristic

# def assign_riders_by_furthest_first(drivers, riders, destination, assignments= None):
#     if assignments is None:
#         assignments = defaultdict(list)

#     drivers = list(drivers)  # Make a copy
#     random.shuffle(drivers)  # Shuffle in place
#     rider_dist_to_dest = {r.name: dist(r.long_lat_pair, destination) for r in riders}
#     shuffled_riders = list(riders)
#     random.shuffle(shuffled_riders)
#     sorted_riders = sorted(shuffled_riders, key=lambda r: rider_dist_to_dest[r.name], reverse=True)

#     unassigned_riders = set()

#     # Normalize whitelist names to lowercase for comparison
#     whitelist_names = set(d.lower() for d in driver_required_riders_to)

#     for rider in sorted_riders:
#         best_driver = None
#         best_insert_pos = None
#         min_marginal_cost = float('inf')

#         for driver in drivers:
#             if driver.amount_seats <= 0 or driver.service_type != rider.service_type:
#                 continue

#             current_route = route_cost(driver.long_lat_pair, assignments[driver], destination)

#             if driver.name.lower() in whitelist_names:
#                 # Try inserting the rider at every possible position
#                 route = assignments[driver]

#                 for i in range(len(route) + 1):
#                     new_route = route[:i] + [rider] + route[i:]
#                     new_cost = route_cost(driver.long_lat_pair, new_route, destination)
#                     detour = new_cost - current_route
#                     if detour < min_marginal_cost:
#                         min_marginal_cost = detour
#                         best_driver = driver
#                         best_insert_pos = i
#             else:
#                 # Use original append-at-end heuristic
#                 new_route = assignments[driver] + [rider]
#                 new_cost = route_cost(driver.long_lat_pair, new_route, destination)
#                 detour = new_cost - current_route
#                 if detour < min_marginal_cost:
#                     min_marginal_cost = detour
#                     best_driver = driver
#                     best_insert_pos = len(assignments[driver])  # append

#         if best_driver is not None:
#             assignments[best_driver].insert(best_insert_pos, rider)
#             best_driver.amount_seats -= 1
#         else:
#             unassigned_riders.add(rider)

#     return assignments, unassigned_riders

# def assign_from_church(drivers, riders, church_location, assignments= None):
#     if assignments is None:
#         assignments = defaultdict(list)

#     drivers = list(drivers)  # Make a copy
#     random.shuffle(drivers)  # Shuffle in place

#     shuffled_riders = list(riders)
#     random.shuffle(shuffled_riders)
#     sorted_riders = sorted(shuffled_riders, key=lambda r: dist(r.long_lat_pair, church_location), reverse=True)

#     unassigned_riders = set()

#     # Normalize whitelist names to lowercase for comparison
#     whitelist_names = set(d.lower() for d in driver_required_riders_back)

#     for rider in sorted_riders:
#         best_driver = None
#         best_insert_pos = None
#         min_marginal_cost = float('inf')

#         for driver in drivers:
#             if driver.amount_seats <= 0 or driver.plans != rider.plans:
#                 continue

#             current_route = route_cost(church_location, assignments[driver], driver.long_lat_pair)

#             if True or driver.name.lower() in whitelist_names:
#                 # Try inserting at every position
#                 route = assignments[driver]
#                 for i in range(len(route) + 1):
#                     new_route = route[:i] + [rider] + route[i:]
#                     new_cost = route_cost(church_location, new_route, driver.long_lat_pair)
#                     detour = new_cost - current_route
#                     if detour < min_marginal_cost:
#                         min_marginal_cost = detour
#                         best_driver = driver
#                         best_insert_pos = i
#             else:
#                 # Use append-at-end logic
#                 new_route = assignments[driver] + [rider]
#                 new_cost = route_cost(church_location, new_route, driver.long_lat_pair)
#                 detour = new_cost - current_route
#                 # print("driver name", driver.name, [x.name for x in new_route], detour, new_cost, current_route)
#                 if detour < min_marginal_cost:
#                     min_marginal_cost = detour
#                     best_driver = driver
#                     best_insert_pos = len(assignments[driver])

#         if best_driver is not None:
#             assignments[best_driver].insert(best_insert_pos, rider)
#             best_driver.amount_seats -= 1
#         else:
#             unassigned_riders.add(rider)

#     return assignments, unassigned_riders

# Global optimization: Minimize distance of all drivers' paths
def assign_riders_by_furthest_first(drivers, riders, destination, assignments=None):
    if assignments is None:
        assignments = defaultdict(list)

    for d in drivers:
        assignments.setdefault(d, [])

    # Sort drivers by distance to destination (furthest first)
    shuffled_drivers = list(drivers)
    random.shuffle(shuffled_drivers)
    sorted_drivers = sorted(shuffled_drivers, key=lambda d: dist(d.long_lat_pair, destination), reverse=True)

    # Sort riders by distance to destination (furthest first)
    shuffled_riders = list(riders)
    random.shuffle(shuffled_riders)
    sorted_riders = sorted(shuffled_riders, key=lambda r: dist(r.long_lat_pair, destination), reverse=True)

    unassigned_riders = set()
    whitelist_names = set(d.lower() for d in driver_required_riders_to)

    for rider in sorted_riders:
        best_driver = None
        best_insert_pos = None
        min_total_cost = float('inf')

        for driver in sorted_drivers:
            if driver.amount_seats <= 0 or driver.service_type != rider.service_type:
                continue

            route = assignments[driver]
            insert_range = range(len(route) + 1) if driver.name.lower() in whitelist_names else [len(route)]

            for i in insert_range:
                new_route = route[:i] + [rider] + route[i:]
                temp_assignments = dict(assignments)
                for d in sorted_drivers:
                    temp_assignments.setdefault(d, [])
                temp_assignments[driver] = new_route

                total_cost = sum(route_cost(d.long_lat_pair, temp_assignments[d], destination) for d in sorted_drivers)

                if total_cost < min_total_cost:
                    min_total_cost = total_cost
                    best_driver = driver
                    best_insert_pos = i

        if best_driver:
            assignments[best_driver].insert(best_insert_pos, rider)
            best_driver.amount_seats -= 1
        else:
            unassigned_riders.add(rider)

    return assignments, unassigned_riders

def assign_from_church(drivers, riders, church_location, assignments=None):
    if assignments is None:
        assignments = defaultdict(list)

    for d in drivers:
        assignments.setdefault(d, [])

    shuffled_drivers = list(drivers)
    random.shuffle(shuffled_drivers)
    sorted_drivers = sorted(shuffled_drivers, key=lambda d: dist(d.long_lat_pair, church_location), reverse=True)

    shuffled_riders = list(riders)
    random.shuffle(shuffled_riders)
    sorted_riders = sorted(shuffled_riders, key=lambda r: dist(r.long_lat_pair, church_location), reverse=True)

    unassigned_riders = set()
    whitelist_names = set(d.lower() for d in driver_required_riders_back)

    for rider in sorted_riders:
        best_driver = None
        best_insert_pos = None
        min_total_cost = float('inf')

        for driver in sorted_drivers:
            if driver.amount_seats <= 0 or driver.plans != rider.plans:
                continue

            route = assignments[driver]
            insert_range = range(len(route) + 1) if driver.name.lower() in whitelist_names else [len(route)]

            for i in insert_range:
                new_route = route[:i] + [rider] + route[i:]
                temp_assignments = dict(assignments)
                for d in sorted_drivers:
                    temp_assignments.setdefault(d, [])
                temp_assignments[driver] = new_route

                total_cost = sum(route_cost(church_location, temp_assignments[d], d.long_lat_pair) for d in sorted_drivers)

                if total_cost < min_total_cost:
                    min_total_cost = total_cost
                    best_driver = driver
                    best_insert_pos = i

        if best_driver:
            assignments[best_driver].insert(best_insert_pos, rider)
            best_driver.amount_seats -= 1
        else:
            unassigned_riders.add(rider)

    return assignments, unassigned_riders

In [ ]:
for i in range(len(df)):
    name = df[NAME_COLUMN][i].strip()
    pickup_location = df[PICKUP_COLUMN][i].strip()
    service_type = df[SERVICE_TYPE_COLUMN][i].strip()
    plans = df[AFTER_SERVICE_PLANS_COLUMN ][i].strip()
    is_driver_val = df[IS_DRIVER_COLUMN][i].strip()
    oc_address = df[OC_ADDRESS][i].strip()

    if pd.isna(name) or name == "": # There's no response for the name
        continue

    if pickup_location not in location_to_address and (pd.isna(oc_address) or oc_address == "") and not pd.isna(name):
        print(f"\n{name} has an empty address cell on the form. They have a pickup location of off campus but did not specify the address in the correct question\n")
        rider = Rider(name, pickup_location, service_type, plans, None)
        oc_people_w_invalid_address.add(rider)
        continue
    
    # Get address based on pickup location
    if pickup_location in location_to_address:
        address = location_to_address[pickup_location]
    else:
        address = oc_address

    coord = ""
    # Get or fetch long-lat coordinate
    if address in address_coords:
        coord = address_coords[address]
    else:
        print("sending geocode address for:", name, address)
        coord = geocode_address(address)

    if pd.isna(is_driver_val) or str(is_driver_val).strip() == "":
        rider = Rider(name, pickup_location, service_type, plans, address)

        if isinstance(coord, Exception):
            oc_people_w_invalid_address.add(rider)
            continue

        address_coords[address] = coord

        rider.long_lat_pair = coord
        riders.add(rider)
    else:
        driver = Driver(name, PASSENGER_LIMIT, pickup_location, service_type, plans, address)
        if isinstance(coord, Exception):
            oc_people_w_invalid_address.add(driver)
            continue 

        address_coords[address] = coord
        driver.long_lat_pair = coord
        drivers.add(driver)

print(f"\nList of people with invalid addresses: {[person.name for person in oc_people_w_invalid_address]}")
print(address_coords)

sending geocode address for: Oriana Tang 3333 cummins st
GEOCODE_ADDRESS address: 3333 cummins st
Location returned by API for 3333 cummins st, houston, USA is 3333 Cummins St, Houston, TX 77027, USA with coord (29.7360786, -95.4382693)
sending geocode address for: Benjamin Kim 2422 Quenby St
GEOCODE_ADDRESS address: 2422 Quenby St
Location returned by API for 2422 Quenby St, houston, USA is 2422 Quenby St, Houston, TX 77005, USA with coord (29.721719, -95.4159846)
sending geocode address for: dawid zhu 6919 Main St, Houston, TX 77030
GEOCODE_ADDRESS address: 6919 Main St, Houston, TX 77030
Location returned by API for 6919 Main St, Houston, TX 77030, USA is 6919 Main St, Houston, TX 77030, USA with coord (29.7057754, -95.4046275)
sending geocode address for: Yun 7950 N Stadium Dr
GEOCODE_ADDRESS address: 7950 N Stadium Dr
Location returned by API for 7950 N Stadium Dr, houston, USA is 7950 N Stadium Dr, Houston, TX 77030, USA with coord (29.6943665, -95.40903349999999)
sending geocode

In [ ]:
# Updates drivers_back and riders_back with their drivers/riders' plans changed to accomodate for rides
def assign_flexible_plans_first(drivers, riders):
    flexible_drivers = [d for d in drivers if "Flexible" in d.plans]
    non_flexible_drivers = [d for d in drivers if "Flexible" not in d.plans]
    flexible_riders = [r for r in riders if "Flexible" in r.plans]
    non_flexible_riders = [r for r in riders if "Flexible" not in r.plans]

    drivers_by_plan = defaultdict(list)
    for d in non_flexible_drivers:
        drivers_by_plan[d.plans].append(d)

    riders_by_plan = defaultdict(list)
    for r in non_flexible_riders:
        riders_by_plan[r.plans].append(r)

    unassigned_flexible_riders = set(flexible_riders)
    assigned_flexible_drivers = set()

    # Step 1: Assign flexible drivers to plans with unmet demand
    for plan in [BACK_HOME_PLAN, LUNCH_PLAN]:
        rider_count = len(riders_by_plan[plan])
        seat_count = sum(d.amount_seats for d in drivers_by_plan[plan])
        needed_seats = rider_count - seat_count

        if needed_seats > 0:
            for d in flexible_drivers:
                if d in assigned_flexible_drivers:
                    continue
                d.plans = plan
                assigned_flexible_drivers.add(d)
                drivers_by_plan[plan].append(d)
                needed_seats -= d.amount_seats
                if needed_seats <= 0:
                    break

    # Step 2: Remaining flexible drivers default to BACK_HOME_PLAN
    for d in flexible_drivers:
        if d not in assigned_flexible_drivers:
            d.plans = BACK_HOME_PLAN
            drivers_by_plan[d.plans].append(d)

   # Step 3: Assign plan labels to flexible riders based on excess space
    remaining_flexible_riders = list(flexible_riders)

    for plan in [BACK_HOME_PLAN, LUNCH_PLAN]:
        rider_count = len(riders_by_plan[plan])
        seat_count = sum(d.amount_seats for d in drivers_by_plan[plan])
        extra_slots = seat_count - rider_count

        if extra_slots <= 0:
            continue  # no room for flexible riders

        assigned = 0
        for r in remaining_flexible_riders[:]:  # Copy slice to avoid mutation during loop
            if assigned >= extra_slots:
                break
            r.plans = plan
            unassigned_flexible_riders.discard(r)
            remaining_flexible_riders.remove(r)
            assigned += 1
            
    # Step 4: Combine flexible + non-flexible riders, then assign
    assigned_flexible_riders = [r for r in flexible_riders if r not in unassigned_flexible_riders]
    all_unassigned_riders = non_flexible_riders + assigned_flexible_riders
    return all_unassigned_riders, unassigned_flexible_riders, drivers, riders

In [ ]:
# Run this code block and below to randomize assignments (still optimal)

DESTINATION = (29.892500, -95.525675)

# ---------------- TO CHURCH ---------------- #

drivers_to = [copy.deepcopy(d) for d in drivers]
riders_to = [copy.deepcopy(r) for r in riders]

# Step 1: Apply whitelist rules (ignores plans/service match)
whitelist_assignments_to, remaining_drivers_to, remaining_riders_to = assign_whitelisted_groups(
    drivers_to, riders_to, driver_required_riders_to, rider_groups_to
)

# for drivers in whitelist_assignments_to:
#     print(drivers.name)
#     for riders in whitelist_assignments_to[drivers]:
#         print(riders.name)

# Step 2: Assign remaining using marginal cost
assignments_to, unassigned_riders_to = assign_riders_by_furthest_first(
    remaining_drivers_to, remaining_riders_to, destination=DESTINATION, assignments=whitelist_assignments_to
)

# ---------------- FROM CHURCH ---------------- #

# Step 1: Deep copy the drivers and riders *after* plan assignments
drivers_back = [copy.deepcopy(d) for d in drivers]
riders_back = [copy.deepcopy(r) for r in riders]

# Step 2: Assign flexible plans and get updated riders + info
unassigned_riders_back, unassigned_flexible_riders, drivers_back, riders_back = assign_flexible_plans_first(drivers_back, riders_back)

# Step 3: Apply whitelist rules
whitelist_assignments_back, remaining_drivers_back, remaining_riders_back = assign_whitelisted_groups(
    drivers_back, riders_back, driver_required_riders_back, rider_groups_back
)

# Step 4: Assign remaining using greedy heuristic
assignments_back, unassigned_riders_back = assign_from_church(
    remaining_drivers_back, remaining_riders_back, church_location=DESTINATION, assignments=whitelist_assignments_back
)

# assign_remaining_flexible_riders(assignments_back, drivers_back, unassigned_riders_back)

# ---------------- Debugging / Output ---------------- #
def print_assignment(assignments):  # driver → list of riders
    for driver, rider_list in assignments.items():
        print(f"\nDriver: {driver.name} ({driver.pickup_location})")
        for rider in rider_list:
            print(f"  - {rider.name} ({rider.pickup_location})")

print("Assignments TO Church:")
print_assignment(assignments_to)

print("\nAssignments FROM Church:")
print_assignment(assignments_back)


Assignments TO Church:

Driver: Grace Park (Life tower)
  - Yun (Off Campus (indicate location in the next question))
  - Melody Hong (Life tower)
  - JJ Lee (Life tower)
  - dawid zhu (Life tower)

Driver: Oriana Tang (Off Campus (indicate location in the next question))

Driver: Sam Ko (Off Campus (indicate location in the next question))
  - Elie Park (Off Campus (indicate location in the next question))
  - Khang Le (Off Campus (indicate location in the next question))
  - Benjamin Kim (Off Campus (indicate location in the next question))

Assignments FROM Church:

Driver: Grace Park (Life tower)
  - Yun (Off Campus (indicate location in the next question))
  - dawid zhu (Life tower)
  - Benjamin Kim (Off Campus (indicate location in the next question))

Driver: Oriana Tang (Off Campus (indicate location in the next question))
  - Melody Hong (Life tower)

Driver: Sam Ko (Off Campus (indicate location in the next question))
  - JJ Lee (Life tower)
  - Elie Park (Off Campus (indicat

In [ ]:
now_cst = datetime.now(ZoneInfo("America/Chicago"))
formatted_time = now_cst.strftime("%m-%d-%Y %H-%M-%S")

excel_to_filename = f"./maps/rides_to/assignments_{formatted_time}.xlsx"
excel_back_filename = f"./maps/rides_back/assignments_{formatted_time}.xlsx"

def export_assignments_to_excel(rides_to, unassigned_riders_to, rides_from, unassigned_riders_from, output_filename="api_allocate.xlsx"):
    wb = Workbook()
    ws = wb.active
    ws.title = "Ride Assignments"

    def place_assignments(assignments, unassigned_riders, start_col, key_col, sort_key_driver, sort_key_rider,
                          driver_color_key, rider_color_key, label_plan=False, include_rider_keys_in_legend=True):
        col = start_col
        row = 2
        curr_car_count = 1
        max_passengers = 0
        local_used_cols = set()
        used_keys = set()

        sorted_assignments = sorted(assignments.items(), key=lambda item: sort_key_driver(item[0]))

        for driver, riders in sorted_assignments:
            driver_text = f"Driver: {driver.name}"
            driver_cell = ws.cell(row=row, column=col, value=driver_text)
            driver_key = driver_color_key(driver)
            used_keys.add(driver_key)
            fill_color = location_colors.get(driver_key, "FFFFFF")
            driver_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            max_passengers = max(max_passengers, len(riders))

            sorted_riders = sorted(riders, key=sort_key_rider)
            for i, rider in enumerate(sorted_riders):
                rider_text = f"{rider.name}"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider)
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

            if curr_car_count % 5 == 0:
                col = start_col
                row += min(8, max_passengers + 3)
            else:
                col += 1
            curr_car_count += 1

        if unassigned_riders or oc_people_w_invalid_address:
            col += 1
            driver_cell = ws.cell(row=row, column=col, value="UNASSIGNED RIDERS")
            driver_cell.fill = PatternFill(start_color="FFCCCCCC", end_color="FFCCCCCC", fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            sorted_unassigned = sorted(unassigned_riders.union(oc_people_w_invalid_address), key=sort_key_rider)
            for i, rider in enumerate(sorted_unassigned):
                rider_text = f"{rider.name} ({rider.plans})" if label_plan else f"{rider.name}"
                rider_text += f" No valid driver available to assign this driver" if rider in unassigned_riders else f" (invalid off campus address)"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider).strip() 
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

        # Create key
        key_row = 2
        for key in sorted(used_keys):
            key_cell = ws.cell(row=key_row, column=key_col, value=key)
            fill_color = location_colors.get(key, "FFFFFF")
            key_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            print("key", key, fill_color)
            key_row += 1

        return local_used_cols

    # Define sort and color strategies
    sort_by_service_then_pickup = lambda obj: (obj.service_type, obj.pickup_location.split()[0])
    sort_by_pickup = lambda obj: obj.pickup_location.split()[0]
    sort_by_plans = lambda obj: obj.plans
    color_by_pickup = lambda obj: obj.pickup_location.split()[0]
    color_by_plans = lambda obj: obj.plans

    # rides_to (left): include rider pickup locations in key
    used_to = place_assignments(
        rides_to,
        unassigned_riders_to,
        start_col=3,
        key_col=2,
        sort_key_driver=sort_by_service_then_pickup,
        sort_key_rider=sort_by_pickup,
        driver_color_key=color_by_pickup,
        rider_color_key=color_by_pickup,
        label_plan=False,
        include_rider_keys_in_legend=True
    )

    # rides_from (right): do NOT include rider pickup in key
    used_from = place_assignments(
        rides_from,
        unassigned_riders_from,
        start_col=11,
        key_col=10,
        sort_key_driver=sort_by_plans,
        sort_key_rider=sort_by_pickup,
        driver_color_key=color_by_plans,
        rider_color_key=color_by_pickup,
        label_plan=True,
        include_rider_keys_in_legend=False
    )

    for col_index in used_from.union(used_to).union({2, 10}):
        col_letter = get_column_letter(col_index)
        max_length = 0
        for row_cells in ws.iter_rows(min_col=col_index, max_col=col_index):
            for cell in row_cells:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[col_letter].width = max_length + 1


    # Set the base directory (e.g., current directory)
    base_dir = "."

    # Recursively find matching files
    pattern = os.path.join(base_dir, "**", "assignments*.xlsx")
    files_to_delete = glob.glob(pattern, recursive=True)

    # Delete each file
    for file_path in files_to_delete:
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")


    wb.save(excel_to_filename)
    wb.save(excel_back_filename)
    wb.save(f"assignments_{formatted_time}.xlsx")
    print(f"Excel file saved to assignments_{formatted_time}")

In [ ]:
export_assignments_to_excel(
    rides_to=assignments_to,
    unassigned_riders_to=unassigned_riders_to,
    rides_from=assignments_back,
    unassigned_riders_from=unassigned_riders_back,
)

key Life fff4cccc
key Off FFFFFFED
key Back home 💙 FFD9D2E9
key Lunch 💛 FFFCE5CD
key NLK 🧡 FFFFCCA4
Deleted: ./assignments_05-17-2025 22-25-21.xlsx
Deleted: ./maps/rides_back/assignments_05-17-2025 22-25-21.xlsx
Deleted: ./maps/rides_to/assignments_05-17-2025 22-25-21.xlsx
Excel file saved to assignments_05-17-2025 22-33-18


In [ ]:
import re
from openpyxl import load_workbook

def clean_cell_value(value):
    """Remove emojis and non-printable characters."""
    if not isinstance(value, str):
        return value
    # Remove emojis and non-ASCII printable characters
    return re.sub(r'[^\x20-\x7E]', '', value)
def workbook_to_html_colored(ws):
    html = '<table border="1" style="border-collapse: collapse; font-family: sans-serif;">\n'
    for row in ws.iter_rows():
        html += '<tr>'
        for cell in row:
            value = clean_cell_value(cell.value) if cell.value is not None else ''

            # Default to white background
            bgcolor = "FFFFFF"

            # Only apply fill if it's solid
            if cell.fill and cell.fill.fill_type == "solid":
                raw_color = cell.fill.start_color.rgb
                if raw_color:
                    bgcolor = raw_color[-6:]  # Strip alpha if present

            # Bold if cell contains "Driver:"
            is_bold = "font-weight: bold;" if isinstance(value, str) and value.strip().startswith("Driver:") else ""

            html += f'<td style="background-color: #{bgcolor}; padding: 5px; {is_bold}">{value}</td>'
        html += '</tr>\n'
    html += '</table>'
    return html

# Load the file you just saved (or keep the `wb` from earlier)
wb = load_workbook(filename=f"assignments_{formatted_time}.xlsx")
ws = wb.active

html_table = workbook_to_html_colored(ws)

# Save to file
with open("assignments_table.html", "w") as f:
    f.write(html_table)

In [ ]:

# lats, lons = zip(*address_coords.values())

# plt.figure(figsize=(8, 6))
# plt.scatter(lons, lats, c='blue', marker='o')

# for label, (lat, lon) in address_coords.items():
#     plt.text(lon, lat, label, fontsize=8)

# plt.xlabel("Longitude")
# plt.ylabel("Latitude")
# plt.title("Pickup Locations")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
def plot_assignments_map(assignments, address_coords, title):
    plt.figure(figsize=(14, 12))
    cmap = plt.cm.get_cmap('tab10', len(assignments))

    # Assign a color to each driver
    driver_colors = {}
    for idx, driver in enumerate(assignments.keys()):
        driver_colors[driver.name] = cmap(idx)

    # Map coordinates to people grouped by driver (to detect mixed groups)
    coord_group_map = defaultdict(lambda: defaultdict(list))

    for driver, riders in assignments.items():
        color = driver_colors[driver.name]
        driver_coord = address_coords.get(driver.address)
        if not driver_coord:
            print(f"Missing coord for driver: {driver.name} ({driver.address})")
            continue

        coord_group_map[driver_coord][driver.name].append(f"[D] {driver.name}")
        last_coord = driver_coord

        for rider in riders:
            rider_coord = address_coords.get(rider.address)
            if not rider_coord:
                print(f"Missing coord for rider: {rider.name} ({rider.address})")
                continue

            coord_group_map[rider_coord][driver.name].append(rider.name.split()[0])

            # Draw arrow
            plt.arrow(
                last_coord[1], last_coord[0],
                rider_coord[1] - last_coord[1],
                rider_coord[0] - last_coord[0],
                color=color,
                linewidth=0.5,
                head_width=0.002,
                head_length=0.001,
                alpha=0.8,
                length_includes_head=True,
                zorder=1,
            )
            last_coord = rider_coord

    # Plot points and grouped labels with offsets
    for coord, driver_to_people in coord_group_map.items():
        for i, (driver_name, people) in enumerate(driver_to_people.items()):
            label = ", ".join(people)
            color = driver_colors[driver_name]
            offset_x = 0.0007
            offset_y = 0.0005 + i * 0.0005  # slight vertical offset per driver group

            plt.scatter(coord[1], coord[0], c=[color], marker='o', s=40, zorder=3)
            plt.text(coord[1] + offset_x, coord[0] + offset_y, label, fontsize=7, color=color)

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# plot_assignments_map(assignments_to, address_coords, title="Routes TO Church")
# print(assignments_back, address_coords)
# print(address_coords)
# plot_assignments_map(assignments_back, address_coords, title="Routes FROM Church")


In [ ]:
# Plot Folium Maps Code

# Generate distinct driver colors
def get_driver_colors(drivers):
    num_drivers = len(drivers)
    cmap = plt.colormaps['tab20'].resampled(num_drivers)
    driver_colors = {}
    for idx, driver in enumerate(drivers):
        rgb = cmap(idx)
        hex_color = mcolors.to_hex(rgb)
        driver_colors[driver.name] = hex_color
    return driver_colors

# Offset points around base location based on how many total people share it
def add_coord_with_offset(base_coord, offset_index, total_points, base_radius=0.0009):
    if total_points == 1:
        return base_coord
    angle = 2 * math.pi * offset_index / total_points
    lat, lon = base_coord
    delta_lat = base_radius * math.sin(angle)
    delta_lon = base_radius * math.cos(angle)
    return (lat + delta_lat, lon + delta_lon)

def plot_assignments_map_folium(assignments, address_coords, map_center=(29.71, -95.41), zoom_start=13, reverse_arrows=False):
    m = folium.Map(location=map_center, zoom_start=zoom_start)
    driver_colors = get_driver_colors(assignments.keys())

    coord_entities = defaultdict(list)
    for driver, riders in assignments.items():
        coord_entities[driver.address].append(driver)
        for rider in riders:
            coord_entities[rider.address].append(rider)

    coord_counts = defaultdict(int)

    arrow_symbol = '  ➤  ' if reverse_arrows else '  ◀  ' 

    for driver, riders in assignments.items():
        color = driver_colors[driver.name]
        base_driver_coord = address_coords.get(driver.address)
        if not base_driver_coord:
            print(f"[WARN] Missing driver coord: {driver.name} ({driver.address})")
            continue

        # Offset driver marker
        coord_counts[driver.address] += 1
        driver_idx = coord_counts[driver.address] - 1
        total_driver_group = len(coord_entities[driver.address])
        driver_coord = add_coord_with_offset(base_driver_coord, driver_idx, total_driver_group)

        folium.Marker(
            driver_coord,
            icon=folium.DivIcon(
                html=f'''
                    <div style="
                        background-color: {color};
                        color: white;
                        font-size: 12px;
                        font-weight: bold;
                        border-radius: 50%;
                        width: 24px;
                        height: 24px;
                        display: flex;
                        align-items: center;
                        justify-content: center;
                        box-shadow: 0 0 4px rgba(0,0,0,0.5);
                        border: 1px solid black;">
                        D
                    </div>
                '''
            ),
            popup=f"[D] {driver.name}"
        ).add_to(m)

      # Step 1: Initialize path starting with driver
        path_coords = [driver_coord]

        # Step 2: Add rider coordinates to the path
        for rider in riders:
            base_rider_coord = address_coords.get(rider.address)
            if not base_rider_coord:
                print(f"[WARN] Missing rider coord: {rider.name} ({rider.address})")
                continue

            coord_counts[rider.address] += 1
            rider_idx = coord_counts[rider.address] - 1
            total_rider_group = len(coord_entities[rider.address])
            rider_coord = add_coord_with_offset(base_rider_coord, rider_idx, total_rider_group)

            # Rider marker
            folium.Marker(
                rider_coord,
                icon=folium.DivIcon(
                    html=f'''
                        <div style="
                            background-color: white;
                            color: {color};
                            font-size: 12px;
                            font-weight: bold;
                            border-radius: 50%;
                            width: 24px;
                            height: 24px;
                            display: flex;
                            align-items: center;
                            justify-content: center;
                            box-shadow: 0 0 4px rgba(0,0,0,0.5);
                            border: 1px solid {color};">
                            R
                        </div>
                    '''
                ),
                popup=rider.name
            ).add_to(m)

            path_coords.append(rider_coord)  # <-- Append to the path

        # Step 3: Optionally add the church destination
        if reverse_arrows:
            path_coords.append(DESTINATION)
            folium.Marker(DESTINATION, icon=folium.Icon(color='red', icon='plus', prefix='fa'),
                        popup="Destination: Church").add_to(m)

        # Step 4: Draw arrows along the full path
        for i in range(len(path_coords) - 1):
            line = PolyLine([path_coords[i], path_coords[i + 1]], color=color, weight=3, opacity=0.7).add_to(m)

            PolyLineTextPath(
                line,
                arrow_symbol,  # Single arrow only (less clutter)
                repeat=True,
                repeat_distance='5px',  # Add arrows even for short segments
                offset=0,  # Align exactly along path
                attributes={
                    'fill': color,
                    'pointer-events': 'none',
                    'font-size': '14px',
                    'font-weight': 'bold',
                }
            ).add_to(m)

    return m

In [ ]:
def add_buttons_to_map(folium_map, filename, homepage_url="https://duongaaron.github.io/automate-nlf-service-rides/"):
    button_html = f"""
    <style>
    @media (max-width: 600px) {{
        .map-buttons {{
            flex-direction: column;
            top: auto;
            bottom: 10px;
            right: 10px;
        }}
    }}

    .map-buttons {{
        position: fixed;
        top: 10px;
        right: 10px;
        z-index: 9999;
        display: flex;
        gap: 10px;
        flex-wrap: wrap;
    }}

    .map-buttons a button {{
        padding: 0.7em 1.2em;
        font-size: 1em;
        border: none;
        border-radius: 5px;
        cursor: pointer;
        white-space: nowrap;
    }}
    </style>

    <div class="map-buttons">
        <a href="{homepage_url}">
            <button style="background-color: #2196F3; color: white;">
                Back to Homepage
            </button>
        </a>
        <a href="{filename}.xlsx" download>
            <button style="background-color: #4CAF50; color: white;">
                Download Assignments as Excel
            </button>
        </a>
    </div>
    """
    folium_map.get_root().html.add_child(folium.Element(button_html))

In [ ]:
rides_to = plot_assignments_map_folium(assignments=assignments_to, address_coords=address_coords, reverse_arrows=True)
add_buttons_to_map(rides_to, f"assignments_{formatted_time}")
print(f"assignments_{formatted_time}.xlsx")
rides_to

assignments_05-17-2025 22-33-18.xlsx


In [ ]:
rides_back = plot_assignments_map_folium(assignments=assignments_back, address_coords=address_coords, reverse_arrows=False)
add_buttons_to_map(rides_back, f"assignments_{excel_back_filename}")
rides_back

In [ ]:
rides_to.save("maps/rides_to/index.html")
rides_back.save("maps/rides_back/index.html")

# Path to your saved HTML file
rides_to_file_path = os.path.abspath("maps/rides_back/index.html")
rides_back_file_path = os.path.abspath("maps/rides_back/index.html")

# Open in default browser
# import webbrowser
# webbrowser.open(f"file://{rides_to_file_path}")
# webbrowser.open(f"file://{rides_back_file_path}")

In [ ]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("index.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Find the first <h1> element and update its contents
if soup.h1:
    soup.h1.string = f"Click on the links to show the map of partitions for this week's driver/rider assignments. Updated {formatted_time} CST"

# Write the modified HTML back
with open("index.html", "w", encoding="utf-8") as f:
    f.write(str(soup))

In [ ]:
excel_filename = f"assignments_{formatted_time}.xlsx"

index_html = f"""
<h1>Click on the links to show the map of partitions for this week's driver/rider assignments. Updated {formatted_time} CST</h1>
<ul>
<li><a href="./maps/rides_to/">Rides to map</a></li>
<li><a href="./maps/rides_back/">Rides back map</a></li>
</ul>

<div style="z-index: 9999; display: flex; gap: 10px;">
    <a href="{excel_filename}" download>
        <button style="margin: 10px; padding: 10px 20px; font-size: 14px; background-color: #4CAF50; color: white; border: none; border-radius: 5px;">
            Download Assignments as Excel
        </button>
    </a>
</div>

<div style="overflow-x: auto; -webkit-overflow-scrolling: touch; touch-action: pan-x">
<iframe
  src="assignments_table.html"
  width="100%"
  height="2000"
  style="min-width: 1000px; border: 1px solid #ccc;"
  scrolling="yes"
></iframe>
</div>
"""

with open("index.html", "w") as f:
    f.write(index_html)

In [ ]:
for addr, coord in address_coords.items():
    print(addr, coord)


3333 cummins st (29.7360786, -95.4382693)
2422 Quenby St (29.721719, -95.4159846)
6919 Main St, Houston, TX 77030 (29.7057754, -95.4046275)
7950 N Stadium Dr (29.6943665, -95.40903349999999)
2022 McClendon (29.7087774, -95.4059077)
2600 Gramercy St (29.7046526, -95.4190738)


In [ ]:
end_time = time.time()

print(f'Total time: {end_time - start_time}')

Total time: 5.378385066986084
